# Fashion MNIST Demo

In this demo, we load the Fashion MNIST dataset and attempt to classify garment classes.  

## Model definition and training  

We define the model architecture, load the dataset, train the model, and save it to disk.  

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# Define the CNN model
class FashionMNISTModel(nn.Module):
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Transform and DataLoader
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, loss function, and optimizer
model = FashionMNISTModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Save the model
torch.save(model.state_dict(), "fashion_mnist_cnn.pth")

print("Model saved as fashion_mnist_cnn.pth")


100%|██████████| 26421880/26421880 [00:04<00:00, 5766576.43it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 3054099.83it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5639992.58it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 7814794.42it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Epoch [1/10], Loss: 0.5380
Epoch [2/10], Loss: 0.3165
Epoch [3/10], Loss: 0.2644
Epoch [4/10], Loss: 0.2343
Epoch [5/10], Loss: 0.2116
Epoch [6/10], Loss: 0.1942
Epoch [7/10], Loss: 0.1728
Epoch [8/10], Loss: 0.1604
Epoch [9/10], Loss: 0.1435
Epoch [10/10], Loss: 0.1329
Model saved as fashion_mnist_cnn.pth


## Model performance evaluation  

We measure accuracy, precision, and recall.  

In [4]:
# !pip install -U scikit-learn #un-comment if needed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 28.3 MB/s eta 0:00:0000:0100:01


In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the saved model
model = FashionMNISTModel().to(device)
model.load_state_dict(torch.load("fashion_mnist_cnn.pth"))
model.eval()

# Lists to store true and predicted labels
true_labels = []
pred_labels = []

# Disable gradient calculation for testing
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(predicted.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='weighted')
recall = recall_score(true_labels, pred_labels, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Accuracy: 0.9144
Precision: 0.9154
Recall: 0.9144


## Improved version  

* Data augmentation: we flip/rotate some images to get more samples and decrease chances of overfitting.  
* Increase dropout rate: again, to prevent overfitting.  
* Learning Rate Scheduler: decrease the learning rate every 5 epochs to prevent over-momentum in our training process.  
* Increase epochs: from 10 to 20. More training with more overfitting protection should be a balanced training routine.  
* Regularization: we add weight decay to the optimizer as a regularization method (again, to prevent overfitting).  


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Define the CNN model
class FashionMNISTModel(nn.Module):
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 20

# Transform and DataLoader with Data Augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
]))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, loss function, and optimizer
model = FashionMNISTModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Save the model
torch.save(model.state_dict(), "fashion_mnist_cnn_improved.pth")
print("Model saved as fashion_mnist_cnn_improved.pth")

# Load the saved model
model = FashionMNISTModel().to(device)
model.load_state_dict(torch.load("fashion_mnist_cnn_improved.pth"))
model.eval()

# Lists to store true and predicted labels
true_labels = []
pred_labels = []

# Disable gradient calculation for testing
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(predicted.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='weighted')
recall = recall_score(true_labels, pred_labels, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Epoch [1/20], Loss: 0.6611
Epoch [2/20], Loss: 0.4142
Epoch [3/20], Loss: 0.3582
Epoch [4/20], Loss: 0.3245
Epoch [5/20], Loss: 0.3022
Epoch [6/20], Loss: 0.2584
Epoch [7/20], Loss: 0.2485
Epoch [8/20], Loss: 0.2442
Epoch [9/20], Loss: 0.2349
Epoch [10/20], Loss: 0.2365
Epoch [11/20], Loss: 0.2279
Epoch [12/20], Loss: 0.2269
Epoch [13/20], Loss: 0.2263
Epoch [14/20], Loss: 0.2257
Epoch [15/20], Loss: 0.2268
Epoch [16/20], Loss: 0.2252
Epoch [17/20], Loss: 0.2245
Epoch [18/20], Loss: 0.2248
Epoch [19/20], Loss: 0.2240
Epoch [20/20], Loss: 0.2254
Model saved as fashion_mnist_cnn_improved.pth
Accuracy: 0.9161
Precision: 0.9155
Recall: 0.9161


## What happens if we add Batch Normalization ?  

We take the exact same code and add batch normalization to our model to see the change in performance.  
(WARNING: we overwrite the .pth file to save disk space)  

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Define the CNN model with Batch Normalization
class FashionMNISTModel(nn.Module):
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 20

# Transform and DataLoader with Data Augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
]))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, loss function, and optimizer
model = FashionMNISTModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Save the model
torch.save(model.state_dict(), "fashion_mnist_cnn_improved.pth")
print("Model saved as fashion_mnist_cnn_improved.pth")

# Load the saved model
model = FashionMNISTModel().to(device)
model.load_state_dict(torch.load("fashion_mnist_cnn_improved.pth"))
model.eval()

# Lists to store true and predicted labels
true_labels = []
pred_labels = []

# Disable gradient calculation for testing
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(predicted.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='weighted')
recall = recall_score(true_labels, pred_labels, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Epoch [1/20], Loss: 0.5274
Epoch [2/20], Loss: 0.3605
Epoch [3/20], Loss: 0.3166
Epoch [4/20], Loss: 0.2916
Epoch [5/20], Loss: 0.2728
Epoch [6/20], Loss: 0.2269
Epoch [7/20], Loss: 0.2182
Epoch [8/20], Loss: 0.2102
Epoch [9/20], Loss: 0.2070
Epoch [10/20], Loss: 0.2011
Epoch [11/20], Loss: 0.1973
Epoch [12/20], Loss: 0.1950
Epoch [13/20], Loss: 0.1936
Epoch [14/20], Loss: 0.1939
Epoch [15/20], Loss: 0.1919
Epoch [16/20], Loss: 0.1908
Epoch [17/20], Loss: 0.1904
Epoch [18/20], Loss: 0.1897
Epoch [19/20], Loss: 0.1916
Epoch [20/20], Loss: 0.1903
Model saved as fashion_mnist_cnn_improved.pth
Accuracy: 0.9251
Precision: 0.9248
Recall: 0.9251


## Conclusion  

Adding batch normalization both decreases the error score during training, and improves measured accuracy, precision, and recall.  

Data augmentation and additional training steps, combined with dropout and regularization by weight decay both increase the eval scores, and decrease overfitting.  

Given the low error scores in the first model training, and relatively high error rates in eval mode, we can say that the first model was overfitted.  
